In [ ]:
# Install google cloud storage package if you haven't already
!pip install google-cloud-storage==2.0.0

In [ ]:
# Upload GCS key to file system
from google.colab import files
uploaded = files.upload()

In [ ]:
# Verify that the key was uploaded
!ls /content

In [ ]:
# Import necessary libraries
import os
import torch
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from google.cloud import storage
from io import BytesIO
import datetime

In [ ]:
# Use a service account key for long-life credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/my-gcs-key.json"  # Replace with your service account key path

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Set the device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

client = storage.Client()
bucket_name = 'vindr-mammo-dataset'  # Replace with your bucket name
bucket = client.bucket(bucket_name)

# Define the custom dataset class
class VindrMammoDataset(Dataset):
    def __init__(self, bucket, dataframe, transform=None):
        self.bucket = bucket        # Google Cloud Storage bucket
        self.dataframe = dataframe  # Dataframe containing image filenames and bi-rads labels
        self.transform = transform  # Transform for data augmentation

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_id = self.dataframe.iloc[idx, 0]
        breast_birads = self.dataframe.iloc[idx, 1]

        # Extract the number from the BI-RADS rating
        birads_value = int(breast_birads.split()[-1])
        # Map BI-RADS values to binary classes
        if birads_value in [1, 2, 3]:  # Benign
            label = 0
        else:  # Malignant (BI-RADS 4, 5)
            label = 1

        # Concatenate the path to the image file in GCS bucket
        img_path = f"images/{image_id}.png"

        # Load the image from the GCS bucket
        blob = self.bucket.blob(img_path)
        image_data = blob.download_as_bytes()
        image = Image.open(BytesIO(image_data)).convert('RGB') # Ensure that it's RGB

        # Apply transformations (if there are any)
        if self.transform:
            image = self.transform(image)

        return image, label


# Define the image transformations for dynamic preprocessing as data is loaded
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Ensure that images are 256x256
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

# Load the finding_annotations.csv file from GCS
csv_blob = bucket.blob("finding_annotations.csv")
csv_data = csv_blob.download_as_text()
annotations_df = pd.read_csv(BytesIO(csv_data.encode()))

# Filter the DataFrame for training and test sets
train_df = annotations_df[annotations_df['split'] == 'training']
test_df = annotations_df[annotations_df['split'] == 'test']

# Create a new DataFrame with only the necessary columns
train_df = train_df[['image_id', 'breast_birads']]
test_df = test_df[['image_id', 'breast_birads']]

# Reset the index for both DataFrames
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Display the first few rows of the training DataFrame
print("Training DataFrame:")
print(train_df.head())

# Create the datasets
train_dataset = VindrMammoDataset(bucket=bucket, dataframe=train_df, transform=transform)
test_dataset = VindrMammoDataset(bucket=bucket, dataframe=test_df, transform=transform)

# Create the data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


## VGG16

In [ ]:
# Build pre-trained VGG16 model
def build_vgg_model(num_classes=2):  # Binary classification
    vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

    # Replace the final classification layer
    vgg16.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)
    return vgg16

vgg16 = build_vgg_model(num_classes=2)  # Set num_classes to 2 for binary classification

# Move the model to the appropriate device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16 = vgg16.to(device)


In [ ]:
# Define the loss function and optimizer, the optimizer requires model parameters and a learning rate. 0.0001 is typical
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.parameters(), lr=1e-4)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    vgg16.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    # Save the model after each epoch for performance evaluation and comparison
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    vgg16_model_path = f"vgg16_breast_cancer_detect_{timestamp}.pth"  # Define the path to save the model
    torch.save(vgg16.state_dict(), vgg16_model_path)  # Save the model's weights

    print(f"Model saved after epoch {epoch+1} to {vgg16_model_path}")


In [ ]:
# Initial Accuracy Evaluation
vgg16.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(valid_loader, desc='Evaluating'):
        images, labels = images.to(device), labels.to(device)
        outputs = vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')


## MobileNetV2

In [ ]:
# Build pre-trained MobileNetV2 model
def build_mobilenetv2_model(num_classes=2):  # Binary classification
    mobilenetv2 = models.mobilenet_v2(pretrained=True)

    # Replace the final classification layer
    mobilenetv2.classifier[1] = nn.Linear(in_features=1280, out_features=num_classes)
    return mobilenetv2

mobilenetv2 = build_mobilenetv2_model(num_classes=2)  # Set num_classes to 2 for binary classification

# Move the model to the appropriate device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mobilenetv2 = mobilenetv2.to(device)


In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenetv2.parameters(), lr=1e-4)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    mobilenetv2.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = mobilenetv2(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Release variables from GPU memory to avoid unnecessary GPU usage - reduce cost and compute power.
torch.cuda.empty_cache()

# Save the model for performance evaluation and comparison
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
mobilenetv2_model_path = f"mobilenetv2_breast_cancer_detect_{timestamp}.pth"  # Define the path to save the model
torch.save(mobilenetv2.state_dict(), mobilenetv2_model_path)  # Save the model's weights

print(f"Model saved to {mobilenetv2_model_path}")


In [ ]:
# Initial Accuracy Evaluation
mobilenetv2.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(valid_loader, desc='Evaluating'):
        images, labels = images.to(device), labels.to(device)
        outputs = mobilenetv2(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')


## ResNet18

In [ ]:
# Build pre-trained ResNet18 model
def build_resnet_model(num_classes=2):  # Binary classification
    resnet18 = models.resnet18(pretrained=True)

    # Replace the final classification layer to match the binary classification task
    resnet18.fc = nn.Linear(in_features=resnet18.fc.in_features, out_features=num_classes)
    return resnet18

resnet18 = build_resnet_model(num_classes=2)  # Set num_classes to 2 for binary classification

# Move the model to the appropriate device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)


In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=1e-4)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    resnet18.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet18(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Save the model for performance evaluation and comparison
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
resnet18_model_path = f"resnet18_breast_cancer_detect_{timestamp}.pth"  # Define the path to save the model
torch.save(resnet18.state_dict(), resnet18_model_path)  # Save the model's weights

print(f"Model saved to {resnet18_model_path}")


In [ ]:
# Initial Accuracy Evaluation
resnet18.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(valid_loader, desc='Evaluating'):
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')